### Projeto de conclusão do módulo III - Técnicas de Programação
#### Elaborado por Maína Alexandre

In [1162]:
import pandas as pd
from geopy.distance import geodesic
from unidecode import unidecode
import re

In [1163]:
# Carregando os arquivos .csv para dataframes
df_escolas = pd.read_csv('escolas.csv')
df_material = pd.read_csv('material_didatico.csv')
df_subprefeituras = pd.read_csv('subprefeituras.csv')

In [1164]:
df_escolas.head()

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


In [1165]:
df_material.head()

,id,Quantidade
0,178,20
1,634,121
2,483,220
3,476,190
4,132,67


In [1166]:
df_subprefeituras.head()

,nome,subprefeitura
0,Santo Cristo,Centro
1,Gamboa,Centro
2,Saúde,Centro
3,Caju,Centro
4,Centro,Centro


In [1167]:
# renomeando a coluna no df de subprefeituras para usar como chave comum no merge
df_subprefeituras.rename(columns={'nome': 'BAIRRO'}, inplace=True)

# tratando o nome dos bairros nos dois dataframes
df_subprefeituras['BAIRRO'] = df_subprefeituras['BAIRRO'].str.upper()
df_escolas['BAIRRO'] = df_escolas['BAIRRO'].str.upper()

In [1168]:
# unindo escolas com subprefeituras
df_junto = df_escolas.merge(df_subprefeituras, on='BAIRRO', how='left')
df_junto.head()

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",Centro
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",Centro
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579",Centro
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456",Centro
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579",Centro


In [1169]:
# unindo com materiais
df_entregas = df_junto.merge(df_material, on='id', how='left')
df_entregas.head()

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura,Quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326",Centro,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202",Centro,121
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579",Centro,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,Praça Argentina 20,"-22,897629","-43,227456",Centro,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579",Centro,67


In [1170]:
df_entregas.columns

Index(['id', 'Escolas_Postos', 'BAIRRO', 'ENDEREÇO ', 'lat', 'lon',
       'subprefeitura', 'Quantidade'],
      dtype='object')

In [1171]:
df_entregas['Escolas_Postos'] = df_entregas['Escolas_Postos'].str.upper()
df_entregas['ENDEREÇO '] = df_entregas['ENDEREÇO '].str.upper()
df_entregas['subprefeitura'] = df_entregas['subprefeitura'].str.upper()


In [1172]:
df_entregas.shape

(152, 8)

In [1173]:
df_entregas.isnull().sum()

id                 0
Escolas_Postos     0
BAIRRO             0
ENDEREÇO           0
lat                0
lon                0
subprefeitura     18
Quantidade         4
dtype: int64

In [1174]:
df_entregas[df_entregas.isnull().any(axis=1)]

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura,Quantidade
10,387,CIEP PRESIDENTE AGOSTINHO NETO,HUMAITA,RUA VISCONDE DE SILVA S/Nº,"-22,959167","-43,199723",NaN,456
30,280,ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO,MARACANÃ/ VILA ISABEL,"R. OITO DE DEZEMBRO, 275","-22,909823","-43,239393",NaN,491
31,60,ESCOLA MUNICIPAL BARÃO HOMEM DE MELO,MARACANÃ/ TIJUCA,"R. ALM. JOÃO CÂNDIDO BRASIL, 352","-22,916829","-43,236925",NaN,'NaN'
38,429,EM FELIX PACHECO,PIEDADE,"R. ASSIS CARNEIRO, 649","-22,895086","-43,313826",ZONA NORTE,NaN
41,373,EM PROFESSOR AUGUSTO PAULINO,MEIER,RUA CÔNEGO TOBIAS 116,"-22,904163","-43,28033",NaN,44
48,135,EM DOMINGOS BEBIANO,INHAUMA,ESTR. ADHEMAR BEBIANO 1870,"-22,873892","-43,277712",NaN,456
50,440,EM JOAQUIM RIBEIRO,INHAUMA,PÇA. SOLDADO COSME DOS SANTOS 28,"-22,87757","-43,285628",NaN,530
86,412,EM MOZART LAGO,OSWALDO CRUZ,RUA JOSÉ CARVALHO SALGADO S/N.º,"-22,872211","-43,350334",NaN,'395'
92,570,EM HELENA LOPES ABRANCHES,GARDENIA AZUL,RUA ACAPORI 495,"-22,96072","-43,353437",NaN,466
93,528,CIEP MARGARET MEE,RECREIO,RUA MINISTRO ALIOMAR BALEEIRO S/Nº,"-23,019811","-43,481721",NaN,51


In [1175]:
df_entregas['Quantidade'].unique()

array(['20', '121', '220', '190', '67', '48', '92', '578', '591', '473',
       '456', '223', '420', '573', "''", '516', '319', '308', '111',
       '525', '445', '164', '293', '399', '185', '576', '63', '491',
       "'NaN'", '336', '159', '342', '262', '440', '365', nan, '183',
       '44', '109', '350', '255', '457', '493', '489', '504', '530',
       '158', '123', '468', '234', '24', '151', '564', '61', '426', '167',
       '509', '292', '210', '153', "'397'", '561', '180', '257', '260',
       '317', '345', '513', '416', '352', '393', '136', "'395'", '115',
       '534', '152', '430', '466', '51', '26', '94', '128', '520', '392',
       '279', '161', '179', '194', '599', '315', '435', '400', '148',
       '269', '571', '235', '187', '362', '71', '221', '372', '479', '99',
       '157', '296', '330', '316', '106', '207', '459', '537', '162',
       '579', '517', "' '", '135', '341', '329', '570', '320', '335',
       '521'], dtype=object)

In [1176]:
df_entregas['Quantidade'] =  df_entregas['Quantidade'].apply(lambda x: x.strip("' ").strip() if isinstance(x, str) else x)
df_entregas['Quantidade'] =  df_entregas['Quantidade'].fillna(0)
df_entregas['Quantidade'] =  df_entregas['Quantidade'].replace({'NaN':0, '':0})
df_entregas['Quantidade'] =  df_entregas['Quantidade'].astype(int)

df_entregas['Quantidade'].unique()



array([ 20, 121, 220, 190,  67,  48,  92, 578, 591, 473, 456, 223, 420,
       573,   0, 516, 319, 308, 111, 525, 445, 164, 293, 399, 185, 576,
        63, 491, 336, 159, 342, 262, 440, 365, 183,  44, 109, 350, 255,
       457, 493, 489, 504, 530, 158, 123, 468, 234,  24, 151, 564,  61,
       426, 167, 509, 292, 210, 153, 397, 561, 180, 257, 260, 317, 345,
       513, 416, 352, 393, 136, 395, 115, 534, 152, 430, 466,  51,  26,
        94, 128, 520, 392, 279, 161, 179, 194, 599, 315, 435, 400, 148,
       269, 571, 235, 187, 362,  71, 221, 372, 479,  99, 157, 296, 330,
       316, 106, 207, 459, 537, 162, 579, 517, 135, 341, 329, 570, 320,
       335, 521])

In [1177]:
print('Média:', df_entregas['Quantidade'].mean())
print('Mediana:', df_entregas['Quantidade'].median())


Média: 292.61842105263156
Mediana: 311.5


In [1178]:
df_entregas['Quantidade'] =  df_entregas['Quantidade'].replace(0, 311)
df_entregas['Quantidade'].unique()

array([ 20, 121, 220, 190,  67,  48,  92, 578, 591, 473, 456, 223, 420,
       573, 311, 516, 319, 308, 111, 525, 445, 164, 293, 399, 185, 576,
        63, 491, 336, 159, 342, 262, 440, 365, 183,  44, 109, 350, 255,
       457, 493, 489, 504, 530, 158, 123, 468, 234,  24, 151, 564,  61,
       426, 167, 509, 292, 210, 153, 397, 561, 180, 257, 260, 317, 345,
       513, 416, 352, 393, 136, 395, 115, 534, 152, 430, 466,  51,  26,
        94, 128, 520, 392, 279, 161, 179, 194, 599, 315, 435, 400, 148,
       269, 571, 235, 187, 362,  71, 221, 372, 479,  99, 157, 296, 330,
       316, 106, 207, 459, 537, 162, 579, 517, 135, 341, 329, 570, 320,
       335, 521])

In [1179]:
df_entregas['BAIRRO'].unique()

array(['CAJU', 'BENFICA', 'CENTRO', 'SÃO CRISTÓVÃO', 'RIO COMPRIDO',
       'IPANEMA', 'SANTA TERESA', 'HUMAITA', 'CATETE', 'FLAMENGO',
       'COPACABANA', 'BOTAFOGO', 'URCA', 'LARANJEIRAS', 'TIJUCA',
       'ANDARAÍ', 'PRAÇA DA BANDEIRA', 'GRAJAÚ', 'VILA ISABEL',
       'MARACANÃ/ VILA ISABEL', 'MARACANÃ/ TIJUCA', 'VIDIGAL',
       'SÃO CONRADO', 'TODOS OS SANTOS', 'LEBLON', 'PILARES',
       'ÁGUA SANTA', 'PIEDADE', 'RIACHUELO', 'MEIER', 'ENGENHO DE DENTRO',
       'ENGENHO NOVO', 'JACARÉ', 'DEL CASTILHO', 'INHAUMA',
       'PARADA DE LUCAS', 'MARIA DA GRAÇA', 'MANGUINHOS',
       'JARDIM AMÉRICA', 'PENHA CIRCULAR', 'OLARIA', 'CAMPO GRANDE',
       'PENHA', 'RAMOS', 'VIGÁRIO GERAL', 'BONSUCESSO', 'MADUREIRA',
       'ACARI', 'CACUIA', 'JARDIM GUANABARA', 'PARQUE COLÚMBIA', 'PAVUNA',
       'ANCHIETA', 'GUADALUPE', 'TURIAÇÚ', 'VILA KOSMOS', 'CAVALCANTI',
       'VILA DA PENHA', 'REALENGO', 'OSWALDO CRUZ', 'ROCHA MIRANDA',
       'CASCADURA', 'BENTO RIBEIRO', 'COLÉGIO', 'CIDADE DE DEU

In [1180]:
pd.set_option('display.float_format', '{:.5f}'.format)
df_entregas['lat'] =  df_entregas['lat'].str.replace(',', '.').astype(float).round(5)
df_entregas['lat'].unique()

array([-2.2880890e+01, -2.2889570e+01, -2.2909290e+01, -2.2897630e+01,
       -2.2924410e+01, -2.2983330e+01, -2.2915540e+01, -2.2907120e+01,
       -2.2898490e+01, -2.2917230e+01, -2.2959170e+01, -2.2922780e+01,
       -2.2940710e+01, -2.2968650e+01, -2.2974830e+01, -2.2748280e+01,
       -2.2981960e+01, -2.2952710e+01, -2.2953820e+01, -2.2945560e+01,
       -2.2933990e+01, -2.2921270e+01, -2.2926860e+01, -2.2915450e+01,
       -2.2918540e+01, -2.2916150e+01, -2.2926840e+01, -2.2929060e+01,
       -2.2909820e+01, -2.2916830e+01, -2.2993700e+01, -2.2997420e+01,
       -2.2892030e+01, -2.2981560e+01, -2.2881340e+01, -2.2906270e+01,
       -2.2895090e+01, -2.2900700e+01, -2.2904160e+01, -2.2900340e+01,
       -2.2903030e+01, -2.2887660e+01, -2.2894180e+01, -2.2878360e+01,
       -2.2885320e+01, -2.2873890e+01, -2.2815270e+01, -2.2877570e+01,
       -2.2884180e+01, -2.2880990e+01, -2.2809450e+01, -2.2839410e+01,
       -2.2847790e+01, -2.2895580e+01, -2.2845630e+01, -2.2835670e+01,
      

In [1181]:
df_entregas['lon'] =  df_entregas['lon'].str.replace(',', '.').astype(float).round(5)
df_entregas['lon'].unique()

array([-4.3225330e+01, -4.3236200e+01, -4.3183580e+01, -4.3227460e+01,
       -4.3208580e+01, -4.3199040e+01, -4.3191590e+01, -4.3195070e+01,
       -4.3237760e+01, -4.3211040e+01, -4.3199720e+01, -4.3176680e+01,
       -4.3176470e+01, -4.3181140e+01, -4.3189060e+01, -4.3213580e+01,
       -4.3191760e+01, -4.3168880e+01, -4.3186590e+01, -4.3181010e+01,
       -4.3228000e+01, -4.3254910e+01, -4.3211090e+01, -4.3265090e+01,
       -4.3247050e+01, -4.3237890e+01, -4.3232870e+01, -4.3239390e+01,
       -4.3236920e+01, -4.3235580e+01, -4.3272180e+01, -4.3279920e+01,
       -4.3217030e+01, -4.3294090e+01, -4.3307920e+01, -4.3313830e+01,
       -4.3255410e+01, -4.3280330e+01, -4.3295940e+01, -4.3267880e+01,
       -4.3302470e+01, -4.3263850e+01, -4.3274890e+01, -4.3254100e+01,
       -4.3277710e+01, -4.3303570e+01, -4.3285630e+01, -4.3262620e+01,
       -4.3253250e+01, -4.3326290e+01, -4.3300070e+01, -4.3261440e+01,
       -4.3570510e+01, -4.3284010e+01, -4.3288160e+01, -4.3268590e+01,
      

In [1182]:
df_entregas[(df_entregas['lat'] > 90) | (df_entregas['lat'] < -90)]

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura,Quantidade
117,206,EM HONDURAS,PRAÇA SECA,PRAÇA BARÃO DA TAQUARA 12,-22896931.00000,-43352667.00000,JACAREPAGUÁ,392


In [1183]:
df_entregas['lat'] = df_entregas['lat'].apply(lambda x: x / 1000000 if x < -90 else x)

In [1184]:
df_entregas[(df_entregas['lon'] > 90) | (df_entregas['lon'] < -90)]

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura,Quantidade
117,206,EM HONDURAS,PRAÇA SECA,PRAÇA BARÃO DA TAQUARA 12,-22.89693,-43352667.00000,JACAREPAGUÁ,392


In [1185]:
df_entregas['lon'] = df_entregas['lon'].apply(lambda x: x / 1000000 if x < -90 else x)

In [1186]:
df_entregas.describe()

,id,lat,lon,Quantidade
count,152.00000,152.00000,152.00000,152.00000
mean,342.63816,-22.90006,-43.36434,311.03289
std,203.30767,0.04894,0.14837,162.35985
min,11.00000,-23.01981,-43.69735,20.00000
25%,160.00000,-22.92685,-43.44749,160.50000
50%,346.50000,-22.89558,-43.33839,313.00000
75%,520.75000,-22.87212,-43.25110,460.75000
max,690.00000,-22.74828,-43.16888,599.00000


In [1187]:
def encontrar_subprefeitura_proxima(bairro):
    if pd.notna(bairro['subprefeitura']):
        return bairro['subprefeitura']

    coordenadas_bairro = (bairro['lat'], bairro['lon'])
    bairros_com_subprefeitura = df_entregas.dropna(subset=['subprefeitura'])
    distancias = bairros_com_subprefeitura.apply(
        lambda x: geodesic(coordenadas_bairro, (x['lat'], x['lon'])).kilometers,
        axis=1
    )
    bairro_mais_proximo = bairros_com_subprefeitura.loc[distancias.idxmin()]
    return bairro_mais_proximo['subprefeitura']

df_entregas['subprefeitura'] = df_entregas.apply(encontrar_subprefeitura_proxima, axis=1)

df_entregas

,id,Escolas_Postos,BAIRRO,ENDEREÇO,lat,lon,subprefeitura,Quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/Nº,-22.88089,-43.22533,CENTRO,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA ÉBANO 187,-22.88957,-43.23620,CENTRO,121
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56",-22.90929,-43.18358,CENTRO,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,PRAÇA ARGENTINA 20,-22.89763,-43.22746,CENTRO,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRAÇA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858,CENTRO,67
...,...,...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTÃO RANGEL,GUARATIBA,ESTRADA DO MAGARÇA 9.183,-22.98046,-43.64354,ZONA OESTE,320
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/Nº",-22.95316,-43.57741,ZONA OESTE,335
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.º,-23.00908,-43.53758,ZONA OESTE,311
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,-22.99412,-43.59368,ZONA OESTE,521


In [1188]:
df_entregas['subprefeitura'].unique()

array(['CENTRO', 'ZONA SUL', 'GRANDE TIJUCA', 'ZONA NORTE', 'ZONA OESTE',
       'ILHAS', 'GRANDE BANGU', 'JACAREPAGUÁ', 'BARRA DA TIJUCA'],
      dtype=object)

In [1189]:
novas_colunas = {
    'Escolas_Postos': 'escola_posto',
    'BAIRRO': 'bairro',
    'ENDEREÇO ': 'endereco',
    'lat': 'latitude',
    'lon': 'longitude',
    'Quantidade': 'quantidade'
}
df_entregas = df_entregas.rename(columns=novas_colunas)
df_entregas.head()

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/Nº,-22.88089,-43.22533,CENTRO,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA ÉBANO 187,-22.88957,-43.23620,CENTRO,121
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56",-22.90929,-43.18358,CENTRO,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SÃO CRISTÓVÃO,PRAÇA ARGENTINA 20,-22.89763,-43.22746,CENTRO,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRAÇA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858,CENTRO,67


In [1190]:
df_entregas['id'].dtype

dtype('int64')

In [1191]:
df_entregas['id'] = df_entregas['id'].astype(str).str.zfill(3)
df_entregas['id'].describe()

count     152
unique    136
top       069
freq        5
Name: id, dtype: object

In [1192]:
df_entregas = df_entregas.drop_duplicates(subset='id', keep='first')

In [1193]:
df_entregas['id'].describe()

count     136
unique    136
top       178
freq        1
Name: id, dtype: object

In [1194]:
df_entregas['escola_posto'] = df_entregas['escola_posto'].apply(lambda x: unidecode(x))
df_entregas['bairro'] = df_entregas['bairro'].apply(lambda x: unidecode(x))
df_entregas['endereco'] = df_entregas['endereco'].apply(lambda x: unidecode(x))
df_entregas['subprefeitura'] = df_entregas['subprefeitura'].apply(lambda x: unidecode(x))

df_entregas


,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533,CENTRO,20
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620,CENTRO,121
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56",-22.90929,-43.18358,CENTRO,220
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746,CENTRO,190
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858,CENTRO,67
...,...,...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUA BELCHIOR DA FONSECA 643,-23.00142,-43.63876,ZONA OESTE,570
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64354,ZONA OESTE,320
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/No",-22.95316,-43.57741,ZONA OESTE,335
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.o,-23.00908,-43.53758,ZONA OESTE,311


In [1195]:
df_entregas[df_entregas['endereco'].str.contains(r'\.')]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56",-22.90929,-43.18358,CENTRO,220
5,017,E.M PRESIDENTE JOSE LINHARES,IPANEMA,"R. BARAO DA TORRE, 90",-22.98333,-43.19904,ZONA SUL,48
9,089,EM MARIO CLAUDIO,RIO COMPRIDO,"R. HADDOCK LOBO, 148",-22.91723,-43.21104,CENTRO,473
13,558,EM CICERO PENA,COPACABANA,"AV. ATLANTICA, 1976",-22.96865,-43.18114,ZONA SUL,573
14,413,EM COCIO BARCELOS,COPACABANA,"R. BARAO DE IPANEMA, 34",-22.97483,-43.18906,ZONA SUL,311
21,035,EM SENADOR CORREA,LARANJEIRAS,R. SEN. CORREA,-22.93399,-43.18101,ZONA SUL,525
23,226,EM AFRANIO PEIXOTO,ANDARAI,"R. FERREIRA PONTES, 328",-22.92686,-43.25491,GRANDE TIJUCA,164
26,126,EM DUQUE DE CAXIAS,GRAJAU,"R. MAL. JOFRE, 74",-22.91854,-43.26509,GRANDE TIJUCA,399
28,163,EM LAUDIMIA TROTTA,TIJUCA,"R. ANTONIO BASILIO, 370",-22.92684,-43.23789,GRANDE TIJUCA,576
30,280,ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO,MARACANA/ VILA ISABEL,"R. OITO DE DEZEMBRO, 275",-22.90982,-43.23939,GRANDE TIJUCA,491


In [1196]:
df_entregas['endereco'] = df_entregas['endereco'].replace('R\.', 'RUA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AV\.', 'AVENIDA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AV ', 'AVENIDA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('PCA\.', 'PRACA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ESTR\.', 'ESTRADA', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ALM\.', 'ALMIRANTE', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('SEN\.', 'SENADOR', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('MAL\.', 'MARECHAL', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('ENG\.', 'ENGENHEIRO', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('SD\.', 'SOLDADO', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('CONJ\.', 'CONJUNTO', regex=True)


In [1197]:
df_entregas[df_entregas['endereco'].str.contains(r'\.')]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
53,041,EM ANDRADE NEVES,JARDIM AMERICA,RUA VIANA DO CASTELO. S/No,-22.80945,-43.32629,ZONA NORTE,468
66,118,CENTRO INTEGRADO DE EDUCACAO PUBLICA ELIS REGINA,BONSUCESSO,RUA PRINCIPAL S/N.o,-22.85783,-43.24045,ZONA NORTE,210
76,256,ESCOLA MUNICIPAL PARAIBA,ANCHIETA,ESTRADA MARECHAL ALENCASTRO 4.035,-22.82636,-43.40288,ZONA NORTE,345
77,091,E.M. PIAUI,GUADALUPE,AVENIDA BRASIL 23.364,-22.84542,-43.37852,ZONA NORTE,311
86,412,EM MOZART LAGO,OSWALDO CRUZ,RUA JOSE CARVALHO SALGADO S/N.o,-22.87221,-43.35033,ZONA NORTE,395
91,016,CIEP JOAO BATISTA DOS SANTOS,CIDADE DE DEUS,"RUA EDGAR WERNECK, 1.565",-22.94711,-43.35974,JACAREPAGUA,430
106,398,EM LEONARDO DA VINCI,BANGU,RUA RENATO REBECCHI S/N.deg,-22.89253,-43.47295,GRANDE BANGU,315
110,576,EM CORYNTHO DA FONSECA,REALENGO,RUA URUQUE S/N.deg,-22.85894,-43.44387,GRANDE BANGU,148
118,687,EM TASSO DA SILVEIRA,REALENGO,RUA GENERAL BERNARDINO DE MATOS S/N.o,-22.88386,-43.41733,GRANDE BANGU,311
132,573,CENTRO INTEGRADO DE EDUCACAO PUBLICA MAJOR MAN...,PACIENCIA,RUA CABO SAULO DE VASCONCELOS S/N.o,-22.88855,-43.63805,ZONA OESTE,207


In [1198]:
df_entregas[df_entregas['endereco'].str.contains(r'\/')]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,-22.88089,-43.22533,CENTRO,20
7,490,E.M TIA CIATA,CENTRO,AVENIDA PRESIDENTE VARGAS S/No,-22.90712,-43.19507,CENTRO,578
10,387,CIEP PRESIDENTE AGOSTINHO NETO,HUMAITA,RUA VISCONDE DE SILVA S/No,-22.95917,-43.19972,ZONA SUL,456
22,011,CIEP SAMUEL WEINER,TIJUCA,"AVENIDAHEITOR BELTRAO, S/No",-22.92127,-43.22800,GRANDE TIJUCA,445
33,653,EM LUCIA MIGUEL PEREIRA,SAO CONRADO,RUA MARTAGAO GESTEIRA S/Ndeg,-22.99742,-43.27218,ZONA SUL,159
35,382,EM SANTOS ANJOS,LEBLON,RUA HUMBERTO DE CAMPOS S/Ndeg,-22.98156,-43.21703,ZONA SUL,262
37,378,EM BRIGADEIRO FARIA LIMA,AGUA SANTA,RUA VIOLETA S/No,-22.90627,-43.30792,ZONA NORTE,365
44,020,EM TAGORE,PIEDADE,PRACA FREDERICO DUVAL S/No,-22.88766,-43.30247,ZONA NORTE,255
45,117,CIEP CHANCELER WILLY BRANDT,JACARE,RUA ALVARO SEIXAS S/No,-22.89418,-43.26385,ZONA NORTE,457
46,502,CIEP PATRICE LUMUMBA,DEL CASTILHO,"PRACA DA CONFEDERACAO SUICA, S/No",-22.87836,-43.27489,ZONA NORTE,493


In [1199]:
df_entregas['endereco'] = df_entregas['endereco'].replace('\. S\/No', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/N\.o', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/N\.deg', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/No', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('S\/Ndeg', 'S/N', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('CASTELOS\/N', 'CASTELO S/N', regex=True)


df_entregas[df_entregas['endereco'].str.contains(r'\/')]


,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/N,-22.88089,-43.22533,CENTRO,20
7,490,E.M TIA CIATA,CENTRO,AVENIDA PRESIDENTE VARGAS S/N,-22.90712,-43.19507,CENTRO,578
10,387,CIEP PRESIDENTE AGOSTINHO NETO,HUMAITA,RUA VISCONDE DE SILVA S/N,-22.95917,-43.19972,ZONA SUL,456
22,011,CIEP SAMUEL WEINER,TIJUCA,"AVENIDAHEITOR BELTRAO, S/N",-22.92127,-43.22800,GRANDE TIJUCA,445
33,653,EM LUCIA MIGUEL PEREIRA,SAO CONRADO,RUA MARTAGAO GESTEIRA S/N,-22.99742,-43.27218,ZONA SUL,159
35,382,EM SANTOS ANJOS,LEBLON,RUA HUMBERTO DE CAMPOS S/N,-22.98156,-43.21703,ZONA SUL,262
37,378,EM BRIGADEIRO FARIA LIMA,AGUA SANTA,RUA VIOLETA S/N,-22.90627,-43.30792,ZONA NORTE,365
44,020,EM TAGORE,PIEDADE,PRACA FREDERICO DUVAL S/N,-22.88766,-43.30247,ZONA NORTE,255
45,117,CIEP CHANCELER WILLY BRANDT,JACARE,RUA ALVARO SEIXAS S/N,-22.89418,-43.26385,ZONA NORTE,457
46,502,CIEP PATRICE LUMUMBA,DEL CASTILHO,"PRACA DA CONFEDERACAO SUICA, S/N",-22.87836,-43.27489,ZONA NORTE,493


In [1200]:
df_entregas['escola_posto'].str[:3].unique()

array(['CEN', 'EM ', 'ESC', 'E.M', 'CIE', 'EM.', 'COL'], dtype=object)

In [1201]:
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('CENTRO INTEGRADO DE EDUCACAO PUBLICA', 'CIEP', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('E\.M.', 'EM ', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('ESCOLA MUNICIPAL', 'EM', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('E\.M', 'EM ', regex=True)
df_entregas['escola_posto'] = df_entregas['escola_posto'].replace('EM\.', 'EM ', regex=True)


In [1202]:
df_entregas['escola_posto'].str[:3].unique()

array(['CIE', 'EM ', 'COL'], dtype=object)

In [1203]:
df_entregas['tipo'] = df_entregas['escola_posto'].str.split().str.get(0)

In [1204]:
df_entregas['tipo'].unique()

array(['CIEP', 'EM', 'COLEGIO'], dtype=object)

In [1205]:
df_entregas

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo
0,178,CIEP HENFIL,CAJU,RUA CARLOS SEIDL S/N,-22.88089,-43.22533,CENTRO,20,CIEP
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,-22.88957,-43.23620,CENTRO,121,EM
2,483,EM CELESTINO SILVA,CENTRO,"RUA DO LAVRADIO, 56",-22.90929,-43.18358,CENTRO,220,EM
3,476,EM FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,-22.89763,-43.22746,CENTRO,190,EM
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,-22.92441,-43.20858,CENTRO,67,EM
...,...,...,...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUA BELCHIOR DA FONSECA 643,-23.00142,-43.63876,ZONA OESTE,570,EM
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,-22.98046,-43.64354,ZONA OESTE,320,EM
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/N",-22.95316,-43.57741,ZONA OESTE,335,EM
149,215,EM NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N,-23.00908,-43.53758,ZONA OESTE,311,EM


In [1206]:
def extrair_ultima_palavra(endereco):
    palavras = endereco.split()
    ultima_palavras = palavras[-1]
    
    if re.match(r'^\d+$', ultima_palavra) or ultima_palavra == 'S/N':
        return ultima_palavra
    else:
        return None

def atualizar_endereco(endereco):
    palavras = endereco.split()
    ultima_palavra = palavras[-1]
    novo_endereco = ' '.join(palavras[:-1])
    
    if re.match(r'^\d+$', ultima_palavra) or ultima_palavra == 'S/N':
        return str(novo_endereco)
    else:
        return endereco

df_entregas['endereco'] = df_entregas['endereco'].str.replace('[.,;]', '', regex=True)
df_entregas['numero'] = df_entregas['endereco'].apply(extrair_ultimo_palavra)
df_entregas['endereco'] = df_entregas['endereco'].apply(atualizar_endereco)

In [1207]:
df_entregas

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero
0,178,CIEP HENFIL,CAJU,RUA CARLOS SEIDL,-22.88089,-43.22533,CENTRO,20,CIEP,S/N
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,-22.88957,-43.23620,CENTRO,121,EM,187
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO,-22.90929,-43.18358,CENTRO,220,EM,56
3,476,EM FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA,-22.89763,-43.22746,CENTRO,190,EM,20
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN,-22.92441,-43.20858,CENTRO,67,EM,45
...,...,...,...,...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUA BELCHIOR DA FONSECA,-23.00142,-43.63876,ZONA OESTE,570,EM,643
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA,-22.98046,-43.64354,ZONA OESTE,320,EM,9183
148,301,EM JONATAS SERRANO,GUARATIBA,ESTRADA DO MATO ALTO,-22.95316,-43.57741,ZONA OESTE,335,EM,S/N
149,215,EM NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO,-23.00908,-43.53758,ZONA OESTE,311,EM,S/N


In [1208]:
df_entregas[df_entregas['numero'].isna()]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero
21,035,EM SENADOR CORREA,LARANJEIRAS,RUA SENADOR CORREA,-22.93399,-43.18101,ZONA SUL,525,EM,None
39,025,EM GEORGE SUMMER,RIACHUELO,RUA ANA NERI,-22.90070,-43.25541,ZONA NORTE,183,EM,None
68,260,EM PROFESSOR JOSUE DE CASTRO,BONSUCESSO,AVENIDA BRASIL 4040 RUA DA VITORIA,-22.87374,-43.24071,ZONA NORTE,397,EM,None
85,520,EM BARONESA DE SAAVEDRA,REALENGO,RUA MARECHAL BIBIANO COSTALAT 483-639,-22.87185,-43.43243,GRANDE BANGU,136,EM,None
95,690,EM FINLANDIA,CAMORIM,RUA IGARAPE-ACU,-22.97199,-43.41011,BARRA DA TIJUCA,94,EM,None
127,534,EM CORA CORALINA,CAMPO GRANDE,RUA VOTORANTIM S/N KM 10 MENDANHA,-22.85878,-43.53705,ZONA OESTE,157,EM,None
139,316,EM PROFESSORA FLAVIA DOS SANTOS SOARES,PACIENCIA,RUA EUDALDO GOMES DA SILVA S/N CONJ URUCANIA,-22.91403,-43.65344,ZONA OESTE,579,EM,None
141,297,EM PROFESSORA SILVIA DE ARAUJO TOLEDO,SANTA CRUZ,AVENIDA CANAL 3 S/N CONJUNTO CESARAO,-22.93011,-43.65290,ZONA OESTE,311,EM,None


In [1209]:
df_entregas['endereco'] = df_entregas['endereco'].replace(' S\/N ', ' ', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace('AVENIDA BRASIL 4040 RUA DA VITORIA', 'AVENIDA BRASIL', regex=True)
df_entregas['endereco'] = df_entregas['endereco'].replace(' 483\-639', '', regex=True)

df_entregas[df_entregas['numero'].isna()]


,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero
21,035,EM SENADOR CORREA,LARANJEIRAS,RUA SENADOR CORREA,-22.93399,-43.18101,ZONA SUL,525,EM,None
39,025,EM GEORGE SUMMER,RIACHUELO,RUA ANA NERI,-22.90070,-43.25541,ZONA NORTE,183,EM,None
68,260,EM PROFESSOR JOSUE DE CASTRO,BONSUCESSO,AVENIDA BRASIL,-22.87374,-43.24071,ZONA NORTE,397,EM,None
85,520,EM BARONESA DE SAAVEDRA,REALENGO,RUA MARECHAL BIBIANO COSTALAT,-22.87185,-43.43243,GRANDE BANGU,136,EM,None
95,690,EM FINLANDIA,CAMORIM,RUA IGARAPE-ACU,-22.97199,-43.41011,BARRA DA TIJUCA,94,EM,None
127,534,EM CORA CORALINA,CAMPO GRANDE,RUA VOTORANTIM KM 10 MENDANHA,-22.85878,-43.53705,ZONA OESTE,157,EM,None
139,316,EM PROFESSORA FLAVIA DOS SANTOS SOARES,PACIENCIA,RUA EUDALDO GOMES DA SILVA CONJ URUCANIA,-22.91403,-43.65344,ZONA OESTE,579,EM,None
141,297,EM PROFESSORA SILVIA DE ARAUJO TOLEDO,SANTA CRUZ,AVENIDA CANAL 3 CONJUNTO CESARAO,-22.93011,-43.65290,ZONA OESTE,311,EM,None


In [1210]:
df_entregas.loc[df_entregas['id'] == '520', 'numero'] = '483-639'
df_entregas.loc[df_entregas['id'] == '260', 'numero'] = '4040'



In [1211]:
df_entregas.loc[df_entregas['id'] == '260']

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero
68,260,EM PROFESSOR JOSUE DE CASTRO,BONSUCESSO,AVENIDA BRASIL,-22.87374,-43.24071,ZONA NORTE,397,EM,4040


In [1212]:
df_entregas[df_entregas['numero'].isna()]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero
21,035,EM SENADOR CORREA,LARANJEIRAS,RUA SENADOR CORREA,-22.93399,-43.18101,ZONA SUL,525,EM,None
39,025,EM GEORGE SUMMER,RIACHUELO,RUA ANA NERI,-22.90070,-43.25541,ZONA NORTE,183,EM,None
95,690,EM FINLANDIA,CAMORIM,RUA IGARAPE-ACU,-22.97199,-43.41011,BARRA DA TIJUCA,94,EM,None
127,534,EM CORA CORALINA,CAMPO GRANDE,RUA VOTORANTIM KM 10 MENDANHA,-22.85878,-43.53705,ZONA OESTE,157,EM,None
139,316,EM PROFESSORA FLAVIA DOS SANTOS SOARES,PACIENCIA,RUA EUDALDO GOMES DA SILVA CONJ URUCANIA,-22.91403,-43.65344,ZONA OESTE,579,EM,None
141,297,EM PROFESSORA SILVIA DE ARAUJO TOLEDO,SANTA CRUZ,AVENIDA CANAL 3 CONJUNTO CESARAO,-22.93011,-43.65290,ZONA OESTE,311,EM,None


In [1213]:
df_entregas['numero'].fillna('S/N', inplace=True)

In [1214]:
df_entregas[df_entregas['numero'].isna()]

,id,escola_posto,bairro,endereco,latitude,longitude,subprefeitura,quantidade,tipo,numero


In [1215]:
df_entregas.rename(columns={'endereco': 'logradouro'}, inplace=True)

In [1216]:
df_entregas

,id,escola_posto,bairro,logradouro,latitude,longitude,subprefeitura,quantidade,tipo,numero
0,178,CIEP HENFIL,CAJU,RUA CARLOS SEIDL,-22.88089,-43.22533,CENTRO,20,CIEP,S/N
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO,-22.88957,-43.23620,CENTRO,121,EM,187
2,483,EM CELESTINO SILVA,CENTRO,RUA DO LAVRADIO,-22.90929,-43.18358,CENTRO,220,EM,56
3,476,EM FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA,-22.89763,-43.22746,CENTRO,190,EM,20
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN,-22.92441,-43.20858,CENTRO,67,EM,45
...,...,...,...,...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,PEDRA DE GUARATIBA,RUA BELCHIOR DA FONSECA,-23.00142,-43.63876,ZONA OESTE,570,EM,643
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA,-22.98046,-43.64354,ZONA OESTE,320,EM,9183
148,301,EM JONATAS SERRANO,GUARATIBA,ESTRADA DO MATO ALTO,-22.95316,-43.57741,ZONA OESTE,335,EM,S/N
149,215,EM NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO,-23.00908,-43.53758,ZONA OESTE,311,EM,S/N


In [1217]:
df_entregas.columns

Index(['id', 'escola_posto', 'bairro', 'logradouro', 'latitude', 'longitude',
       'subprefeitura', 'quantidade', 'tipo', 'numero'],
      dtype='object')

In [1218]:
nova_ordem = ['id', 'escola_posto', 'tipo', 'logradouro', 'numero', 'bairro', 'subprefeitura', 'quantidade', 'latitude', 'longitude']
df_entregas = df_entregas[nova_ordem]
df_entregas

,id,escola_posto,tipo,logradouro,numero,bairro,subprefeitura,quantidade,latitude,longitude
0,178,CIEP HENFIL,CIEP,RUA CARLOS SEIDL,S/N,CAJU,CENTRO,20,-22.88089,-43.22533
1,634,EM ALICE DO AMARAL PEIXOTO,EM,RUA EBANO,187,BENFICA,CENTRO,121,-22.88957,-43.23620
2,483,EM CELESTINO SILVA,EM,RUA DO LAVRADIO,56,CENTRO,CENTRO,220,-22.90929,-43.18358
3,476,EM FLORIANO PEIXOTO,EM,PRACA ARGENTINA,20,SAO CRISTOVAO,CENTRO,190,-22.89763,-43.22746
4,132,EM PEREIRA PASSOS,EM,PRACA CONDESSA PAULO DE FRONTIN,45,RIO COMPRIDO,CENTRO,67,-22.92441,-43.20858
...,...,...,...,...,...,...,...,...,...,...
146,210,EM DEBORA MENDES DE MORAES,EM,RUA BELCHIOR DA FONSECA,643,PEDRA DE GUARATIBA,ZONA OESTE,570,-23.00142,-43.63876
147,474,EM ENGENHEIRO GASTAO RANGEL,EM,ESTRADA DO MAGARCA,9183,GUARATIBA,ZONA OESTE,320,-22.98046,-43.64354
148,301,EM JONATAS SERRANO,EM,ESTRADA DO MATO ALTO,S/N,GUARATIBA,ZONA OESTE,335,-22.95316,-43.57741
149,215,EM NARCISA AMALIA,EM,ESTRADA TEODORETO DE CAMARGO,S/N,ILHA DE GUARATIBA,ZONA OESTE,311,-23.00908,-43.53758
